<h1 style="color:   #F91818 ;"> <b> Лабораторная работа №7 </h1>
<h3 style="color:   #FB6868 ;"> <b> Работа с текстовыми данными </h3>

<h3 style="color:   #F6ACAC ;"> 1. Пример, описанный в ходе работы </h3>

In [50]:
from sklearn.datasets import load_files
reviews_train = load_files("aclImdb/train/")
# load_files возвращает коллекцию, содержащую обучающие тексты и обучающие метки
text_train, y_train = reviews_train.data, reviews_train.target

text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

reviews_test = load_files("aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(text_train)

In [52]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch", max_iter=10, random_state=0)

# Мы строим модель и преобразуем данные в один этап
# Преобразование займет некоторое время,
# и мы можем сэкономить время, выполнив обе операции сразу
document_topics = lda.fit_transform(X)

KeyboardInterrupt: 

In [ ]:
lda.components_.shape

In [ ]:
import numpy as np
# Для каждой темы (строки в components_) сортируем признаки (по возрастанию) # Инвертируем строки с помощью [:, ::-1], чтобы получить сортировку по убыванию
sorting =np.argsort(lda.components_, axis=1)[:, ::-1]
# Получаем имена признаков из векторизатора
feature_names = np.array(vect.get_feature_names_out())

In [ ]:
import mglearn
# Выводим 10 тем:
mglearn.tools.print_topics (topics=range (10), feature_names=feature_names, 
                            sorting=sorting, topics_per_chunk=5, n_words=10)

In [ ]:
lda100 = LatentDirichletAllocation (n_components=100, learning_method="batch", max_iter=10, random_state=0)
document_topics100 = lda100.fit_transform(X)

In [ ]:
import numpy as np
topics = np.array([7,16,24,25,28,36,37,45,51,53,54,63,89,97])

sorting = np.argsort(lda100.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names_out())
mglearn.tools.print_topics(topics=topics, feature_names=feature_names,
                           sorting=sorting,topics_per_chunk=7,n_words=20)

In [ ]:
# сортируем документы по весу темы 45 "музыка"
music = np.argsort(document_topics100[:, 45])[::-1]

# печатаем 10 документов, в которых данная тема является наиболее важной
for i in music[:10]: # выводим первые два предложения
    print (b".".join(text_train[i].split(b".")[:2]) + b".\n")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
topic_names = ["{:>2} ".format(i) + " ".join(words) 
               for i, words in enumerate(feature_names[sorting[:, :2]])]
# two column bar chart:

for col in [0, 1]:

    start = col * 50
    end = (col + 1) * 50
    ax[col].barh(np.arange(50), np.sum(document_topics100, axis=0)[start:end]) 
    ax[col].set_yticks(np.arange(50))
    ax[col].set_yticklabels (topic_names[start:end], ha="left", va="top")
    ax[col].invert_yaxis()
    ax[col].set_xlim(0, 2000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad=130)
plt.tight_layout()

<h3 style="color:   #F6ACAC ;"> 2. Создание «мешка слов», например, набор из 20-25 отзывов о чем-либо и краткое пояснение к данным. </h3>

In [ ]:
# Импорт необходимых библиотек
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
import nltk
import pandas as pd

In [ ]:
# Создание своего «мешка слов»
my_reviews = [
    "Отличный фильм! Очень понравился сюжет и актерская игра.",
    "Не понравился финал, ожидал чего-то более неожиданного.",
    "Сюжет слабый, но визуальная составляющая поражает.",
    "Замечательный фильм для всей семьи, смешно и трогательно.",
    "Фильм оставил двоякое впечатление, некоторые моменты поразили, но есть и неясности.",
    "Отличная режиссерская работа, актеры справились на отлично!",
    "Не смог оторваться от экрана, интригующий сюжет до последней минуты.",
    "Фильм подарил мне кучу положительных эмоций, спасибо за такое кино!",
    "Сюжет затянут, несмотря на это, фильм оставил приятное впечатление.",
    "Эффектные визуальные эффекты, но где-то теряется смысл сюжета.",
    "Ожидал большего, но не получил. Впечатление разочарования.",
    "Фильм для тех, кто ценит глубокий смысл и хорошую игру актеров.",
    "Замечательное сочетание драмы и юмора, смотрится на одном дыхании.",
    "Необычный подход к теме, но не у всех он вызовет восторг.","Фильм, который оставил меня в раздумьях, интересные философские аспекты.",
    "Сценарий поражает своей оригинальностью, необычные повороты сюжета.",
    "Атмосфера фильма так и пронизана таинственностью, что создает неповторимый опыт.",
    "Не для каждого, но любителям тяжелых драм рекомендую.",
    "Зрелищные боевые сцены и отличная работа художников по костюмам.",
    "Неплохой психологический триллер, но финал оставил вопросы.",
    "Фильм смог перенести меня в другой мир, волшебство на экране.",
    "Позитивный настрой и отличный юмор, идеальный выбор для веселого вечера.",
    "Игра актеров на высшем уровне, каждый персонаж - настоящая жемчужина.",
    "Оригинальный подход к жанру, стоит посмотреть для разнообразия.","Фильм, который заставил задуматься о смысле жизни, глубокий и трогательный.",
    "Отличная игра актеров, особенно впечатлил перформанс главного героя.",
    "Сюжет развивается неспешно, но это лишь добавляет загадочности и интриги.",
    "Необычный жанровый микс, создает интересное визуальное восприятие.",
    "Картина о любви, которая оставила слезы на глазах.",
    "Блестящая характерная музыка, идеально сочетающаяся с общей атмосферой.",
    "Фильм, который вызывает смех и грусть, настоящая эмоциональная ачивка.",
    "Необычная концовка, которая оставляет много места для интерпретации.",
    "Браво режиссеру за отличную работу с атмосферой и цветовой палитрой.",
    "Фильм, который нельзя забыть, завораживает своей непредсказуемостью."
]

In [ ]:
# Создание датафрейма
data = pd.DataFrame({"Review": my_reviews})

<h3 style="color:   #F6ACAC ;">3. Создание словаря стоп-слов, применение стоп-слова к «мешку слов»</h3>

In [ ]:
# Создание словаря стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Загрузка ресурса vader_lexicon
nltk.download('vader_lexicon')

# Применение стоп-слов к «мешку слов»
data['Review'] = data['Review'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h3 style="color:   #F6ACAC ;">4. Масштабирование данных с помощью tf-idf</h3>

In [55]:
# Масштабирование данных с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Review'])

# Разделение данных на обучающий и тестовый наборы
X_train, X_test = train_test_split(tfidf_matrix, test_size=0.2, random_state=42)

# Обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, X_train)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.